<p style="align: center;"><img src="https://static.tildacdn.com/tild6636-3531-4239-b465-376364646465/Deep_Learning_School.png" width="400"></p>

# Глубокое обучение. Часть 2
# Домашнее задание по теме "Механизм внимания"

Это домашнее задание проходит в формате peer-review. Это означает, что его будут проверять ваши однокурсники. Поэтому пишите разборчивый код, добавляйте комментарии и пишите выводы после проделанной работы.

В этом задании вы будете решать задачу классификации математических задач по темам (многоклассовая классификация) с помощью Transformer.

В качестве датасета возьмем датасет математических задач по разным темам. Нам необходим следующий файл:

[Файл с классами](https://docs.google.com/spreadsheets/d/1IMRxByfg7gjoZ5i7rxvuNDvSrbdOJOc-/edit?usp=drive_link&ouid=104379615679964018037&rtpof=true&sd=true)

In [1]:
!ls -la /kaggle/input/no-problems/

total 2832
drwxr-xr-x 2 nobody nogroup       0 Apr 13 08:07 .
drwxr-xr-x 3 root   root       4096 Apr 13 08:07 ..
-rw-r--r-- 1 nobody nogroup 2892351 Apr 13 08:07 data_problems.csv


In [2]:
!ls -la /kaggle/working/

total 12
drwxr-xr-x 3 root root 4096 Apr 13 08:07 .
drwxr-xr-x 5 root root 4096 Apr 13 08:07 ..
drwxr-xr-x 2 root root 4096 Apr 13 08:07 .virtual_documents


загрузил эксельку в csv на каггл

In [3]:
import pandas as pd

filepath = "/kaggle/input/no-problems/data_problems.csv"

data = pd.read_csv(filepath)

print(data.head())


   Unnamed: 0                                             Задача   Тема
0           0  Между девятью планетами Солнечной системы введ...  Графы
1           1  В стране Цифра есть 9 городов с названиями 1, ...  Графы
2           2  В государстве 100 городов, и из каждого из них...  Графы
3           3  В классе 30 человек. Может ли быть так, что 9 ...  Графы
4           4  В городе Маленьком 15 телефонов. Можно ли их с...  Графы


In [4]:
# Получаем данные в столбце Тема по индексу
first_column = data.iloc[:, 2]

# Находим уникальные значения
unique_values = first_column.unique()

# Подсчитываем количество уникальных значений
num_unique_values = len(unique_values)

print(f"Количество уникальных значений в столбце Тема: {num_unique_values}")
print(f"они сами: {unique_values}")


Количество уникальных значений в столбце Тема: 7
они сами: ['Графы' 'Геометрия' 'Многочлен' 'Комбинаторика' 'Теория чисел'
 'Инвариант' 'Дирихле']


**Hint:** не перезаписывайте модели, которые вы получите на каждом из этапов этого дз. Они ещё понадобятся.

### Задание 1 (2 балла)

Напишите кастомный класс для модели трансформера для задачи классификации, использующей в качествке backbone какую-то из моделей huggingface.

Т.е. конструктор класса должен принимать на вход название модели и подгружать её из huggingface, а затем использовать в качестве backbone (достаточно возможности использовать в качестве backbone те модели, которые упомянуты в последующих пунктах)

In [5]:
!pip install transformers

In [6]:
import torch
import torch.nn as nn
from transformers import AutoModel
from typing import Union, Dict

print("ok")

ok


In [7]:
# как на странно именно этот класс потом не использую :-)
class TransformerClassificationModel(nn.Module):
    def __init__(self, base_transformer_model: Union[str, nn.Module], num_labels: int, output_attentions=False):
        """
        Инициализация модели классификации на основе трансформера.
        
        Параметры:
        base_transformer_model (Union[str, nn.Module]): Название предобученной модели трансформера из huggingface
                                                        или экземпляр модели nn.Module.
        num_labels (int): Количество классов для классификации.
        output_attentions (bool): Если True, модель будет возвращать слои внимания.
        """
        super(TransformerClassificationModel, self).__init__()
        self.output_attentions = output_attentions
        if isinstance(base_transformer_model, str):
            self.backbone = AutoModel.from_pretrained(base_transformer_model, output_attentions=output_attentions)
        else:
            self.backbone = base_transformer_model
        
        # Добавляем дополнительные слои для классификации.
        self.classifier = nn.Linear(self.backbone.config.hidden_size, num_labels)
        
        # Для сохранения истории потерь и точности
        self.training_losses = []
        self.training_accuracies = []
        self.validation_losses = []
        self.validation_accuracies = []

    def forward(self, input_ids, attention_mask=None, token_type_ids=None) -> Dict[str, torch.Tensor]:
        """
        Прямой проход модели.
        
        Параметры:
        input_ids (torch.Tensor): Тензор идентификаторов токенов.
        attention_mask (torch.Tensor, optional): Тензор масок внимания.
        token_type_ids (torch.Tensor, optional): Тензор типов токенов.

        Возвращает:
        Dict[str, torch.Tensor]: Словарь с логитами предсказаний классов и, при наличии, слои внимания.
        """
        # Пропагация входных данных через backbone.
        outputs = self.backbone(input_ids=input_ids, 
                                attention_mask=attention_mask, 
                                token_type_ids=token_type_ids,
                                output_attentions=self.output_attentions)
        
        # Используем [CLS] токен для классификации.
        cls_token_state = outputs.last_hidden_state[:, 0, :]
        
        # Пропагация через классификационный слой.
        logits = self.classifier(cls_token_state)

        response = {"logits": logits}
        if self.output_attentions:
            response["attentions"] = outputs.attentions

        return response

    def add_metrics(self, train_loss, train_accuracy, val_loss=None, val_accuracy=None):
        self.training_losses.append(train_loss)
        self.training_accuracies.append(train_accuracy)
        if val_loss is not None and val_accuracy is not None:
            self.validation_losses.append(val_loss)
            self.validation_accuracies.append(val_accuracy)

    def get_metrics(self):
        return {
            "training_losses": self.training_losses,
            "training_accuracies": self.training_accuracies,
            "validation_losses": self.validation_losses,
            "validation_accuracies": self.validation_accuracies
        }
    
print("ok")


ok


### Задание 2 (1 балл)

Напишите функцию заморозки backbone у модели (если необходимо, возвращайте из функции модель)

In [8]:
def freeze_backbone_function(model: TransformerClassificationModel):
    # Перебираем все параметры в backbone модели
    for param in model.backbone.parameters():
        # Выключаем вычисление градиентов
        param.requires_grad = False

    # Возвращать модель необязательно, так как изменения произведены "на месте"
    return model

print("ok")

ok


### Задание 3 (2 балла)

Напишите функцию, которая будет использована для тренировки (дообучения) трансформера (TransformerClassificationModel). Функция должна поддерживать обучение с замороженным и размороженным backbone.

In [9]:
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import get_linear_schedule_with_warmup
import copy

def train_transformer(model, data_loader, freeze_backbone=True, epochs=3, learning_rate=5e-5, num_training_steps=None, val_loader=None):
    # Предполагаем, что модель уже содержит методы add_metrics и get_metrics
    optimizer = torch.optim.AdamW(filter(lambda p: p.requires_grad, model.parameters()), lr=learning_rate)
    criterion = torch.nn.CrossEntropyLoss()

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        correct_predictions = 0
        total_predictions = 0

        for batch in data_loader:
            inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
            labels = batch['labels'].to(model.device)

            outputs = model(**inputs)
            loss = criterion(outputs['logits'], labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = torch.max(outputs['logits'], 1)
            correct_predictions += (predicted == labels).sum().item()
            total_predictions += labels.size(0)

        train_loss = total_loss / len(data_loader)
        train_accuracy = correct_predictions / total_predictions

        # Если есть валидационный DataLoader, вычисляем потери и точность для валидации
        if val_loader:
            model.eval()
            val_loss = 0
            val_correct = 0
            val_total = 0
            with torch.no_grad():
                for batch in val_loader:
                    inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
                    labels = batch['labels'].to(model.device)
                    outputs = model(**inputs)
                    loss = criterion(outputs['logits'], labels)
                    val_loss += loss.item()
                    _, predicted = torch.max(outputs['logits'], 1)
                    val_correct += (predicted == labels).sum().item()
                    val_total += labels.size(0)

            val_loss /= len(val_loader)
            val_accuracy = val_correct / val_total
            model.add_metrics(train_loss, train_accuracy, val_loss, val_accuracy)
        else:
            model.add_metrics(train_loss, train_accuracy)

    return model

print("ok")


ok


### Задание 4 (1 балл)

Проверьте вашу функцию из предыдущего пункта, дообучив двумя способами
*cointegrated/rubert-tiny2* из huggingface.

In [10]:
import torch
import torch.nn as nn
from transformers import AutoModel

class RubertTinyClassifier(nn.Module):
    def __init__(self, pretrained_model_name, num_labels, output_attentions=False):
        super(RubertTinyClassifier, self).__init__()
        self.backbone = AutoModel.from_pretrained(pretrained_model_name, output_attentions=output_attentions)
        self.classifier = nn.Linear(self.backbone.config.hidden_size, num_labels)
        self.output_attentions = output_attentions  # Сохраняем параметр output_attentions
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.to(self.device)

        # Инициализация списков для хранения метрик обучения и валидации
        self.training_losses = []
        self.training_accuracies = []
        self.validation_losses = []
        self.validation_accuracies = []

    def forward(self, input_ids, attention_mask=None, token_type_ids=None):
        # Выполняем прямой проход, запрашивая карты внимания при необходимости
        outputs = self.backbone(input_ids=input_ids, 
                                attention_mask=attention_mask, 
                                token_type_ids=token_type_ids,
                                output_attentions=self.output_attentions)
        pooler_output = outputs.pooler_output
        logits = self.classifier(pooler_output)
        
        response = {'logits': logits}
        if self.output_attentions and 'attentions' in outputs:
            response['attentions'] = outputs.attentions
        
        return response

    def add_metrics(self, train_loss, train_accuracy, val_loss=None, val_accuracy=None):
        self.training_losses.append(train_loss)
        self.training_accuracies.append(train_accuracy)
        if val_loss is not None and val_accuracy is not None:
            self.validation_losses.append(val_loss)
            self.validation_accuracies.append(val_accuracy)

    def get_metrics(self):
        return {
            "training_losses": self.training_losses,
            "training_accuracies": self.training_accuracies,
            "validation_losses": self.validation_losses,
            "validation_accuracies": self.validation_accuracies
        }


print("ok")



ok


In [11]:
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer
import torch


In [ ]:
# создаем датасет(ы)
# Шаг 1: Загрузка данных
file_path = '/kaggle/input/no-problems/data_problems.csv'
df = pd.read_csv(file_path)

# Шаг 2: Преобразование категорий в числовой формат
unique_labels = df['Тема'].unique()
label_to_id = {label: id for id, label in enumerate(unique_labels)}
id_to_label = {id: label for label, id in label_to_id.items()}

df['label_id'] = df['Тема'].map(label_to_id)

print("ok")

In [ ]:
# Шаг 3: Создание класса Dataset
class MathProblemsDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_len=512):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe['Задача']
        self.labels = dataframe['label_id']
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, idx):
        text = str(self.text[idx])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=False,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        input_ids = inputs['input_ids'].flatten()
        attention_mask = inputs['attention_mask'].flatten()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': torch.tensor(self.labels[idx], dtype=torch.long)
        }

print("ok")


In [ ]:
# Шаг 4: Токенизация
# pretrained_model_name = название модели, которую будем использовать
pretrained_model_name = 'cointegrated/rubert-tiny2'
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)

# Создание экземпляра Dataset
dataset = MathProblemsDataset(df, tokenizer)



print("ok")


In [ ]:
from torch.utils.data import random_split

# Определяем размеры для разделения
total_count = len(dataset)
train_count = int(0.7 * total_count)  # 70% данных на тренировку
val_count = int(0.15 * total_count)   # 15% данных на валидацию
test_count = total_count - train_count - val_count  # Оставшиеся 15% данных на тест

# Разделим dataset
train_dataset, val_dataset, test_dataset = random_split(dataset, [train_count, val_count, test_count])

# Проверим, что размеры наборов данных верны
print(f"Total: {total_count}, Train: {len(train_dataset)}, Validation: {len(val_dataset)}, Test: {len(test_dataset)}")

print("ok")

In [ ]:
# Создание DataLoader для тренировочного набора
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# Создание DataLoader для валидационного набора
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Создание DataLoader для тестового набора
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

print("ok")

In [ ]:
# будет нужна функция
def evaluate(model, data_loader):
    model.eval()  # Перевод модели в режим оценки
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():  # Отключение вычисления градиентов
        for batch in data_loader:
            inputs = {k: v.to(model.device) for k, v in batch.items() if k != 'labels'}
            labels = batch['labels'].to(model.device)

            outputs = model(**inputs)
            _, predicted = torch.max(outputs['logits'], 1)

            correct_predictions += (predicted == labels).sum().item()
            total_predictions += labels.size(0)

    accuracy = correct_predictions / total_predictions
    return accuracy   


In [ ]:
NUM_LABELS = 7 # в самом начале подсмотрели, сколько там классов

# Инициализация и дообучение с замороженным backbone
rubert_tiny_transformer_model = RubertTinyClassifier(pretrained_model_name="cointegrated/rubert-tiny2", num_labels=NUM_LABELS, output_attentions=True)
rubert_tiny_finetuned_with_freezed_backbone = train_transformer(rubert_tiny_transformer_model, train_loader, freeze_backbone=True, val_loader=val_loader)

metrics_freezed = rubert_tiny_finetuned_with_freezed_backbone.get_metrics()
print(metrics_freezed['training_losses'])
print(metrics_freezed['training_accuracies'])
if 'validation_losses' in metrics_freezed:
    print(metrics_freezed['validation_losses'])
    print(metrics_freezed['validation_accuracies'])
    
test_accuracy = evaluate(rubert_tiny_finetuned_with_freezed_backbone, test_loader)
print(f"Test Accuracy: {test_accuracy}")

# Инициализация и полное дообучение
rubert_tiny_transformer_model = RubertTinyClassifier(pretrained_model_name="cointegrated/rubert-tiny2", num_labels=NUM_LABELS, output_attentions=True)
rubert_tiny_full_finetuned = train_transformer(rubert_tiny_transformer_model, train_loader, freeze_backbone=False, val_loader=val_loader)

metrics_unfreezed = rubert_tiny_full_finetuned.get_metrics()
print(metrics_unfreezed['training_losses'])
print(metrics_unfreezed['training_accuracies'])
if 'validation_losses' in metrics_unfreezed:
    print(metrics_unfreezed['validation_losses'])
    print(metrics_unfreezed['validation_accuracies'])
    
test_accuracy = evaluate(rubert_tiny_full_finetuned, test_loader)
print(f"Test Accuracy: {test_accuracy}")

print("ok")


In [ ]:
# нарисуем графики
import matplotlib.pyplot as plt

# Функция для отрисовки графиков
def plot_metrics(train_losses, train_accuracies, val_losses, val_accuracies):
    epochs = range(1, len(train_losses) + 1)

    plt.figure(figsize=(12, 5))

    # Построение графика потерь
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, 'b-', label='Training Loss')
    plt.plot(epochs, val_losses, 'r-', label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    # Построение графика точности
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracies, 'b-', label='Training Accuracy')
    plt.plot(epochs, val_accuracies, 'r-', label='Validation Accuracy')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.show()


In [ ]:
train_losses = metrics_freezed['training_losses']
train_accuracies = metrics_freezed['training_accuracies']
val_losses = metrics_freezed['validation_losses']
val_accuracies = metrics_freezed['validation_accuracies']
plot_metrics(train_losses, train_accuracies, val_losses, val_accuracies)


train_losses = metrics_unfreezed['training_losses']
train_accuracies = metrics_unfreezed['training_accuracies']
val_losses = metrics_unfreezed['validation_losses']
val_accuracies = metrics_unfreezed['validation_accuracies']
plot_metrics(train_losses, train_accuracies, val_losses, val_accuracies)



In [ ]:
# Сохранение модели после обучения с замороженным backbone
model_path = "/kaggle/working/rubert_tiny_finetuned_with_freezed_backbone.pth"
torch.save(rubert_tiny_finetuned_with_freezed_backbone.state_dict(), model_path)

# Сохранение модели после полного обучения
model_path = "/kaggle/working/rubert_tiny_full_finetuned.pth"
torch.save(rubert_tiny_full_finetuned.state_dict(), model_path)

In [ ]:
import os
from IPython.display import FileLink

# Путь к файлу на вашем ноутбуке Kaggle
file_path = '/kaggle/working/rubert_tiny_finetuned_with_freezed_backbone.pth'

# Проверяем, существует ли файл
if os.path.isfile(file_path):
    # Создаем ссылку для скачивания
    download_link = FileLink(file_path, result_html_prefix="Click here to download: ")
    display(download_link)
else:
    print("Файл не найден")


In [ ]:
# Путь к файлу на вашем ноутбуке Kaggle
file_path = '/kaggle/working/rubert_tiny_full_finetuned.pth'

# Проверяем, существует ли файл
if os.path.isfile(file_path):
    # Создаем ссылку для скачивания
    download_link = FileLink(file_path, result_html_prefix="Click here to download: ")
    display(download_link)
else:
    print("Файл не найден")

### Задание 5 (1 балл)

Обучите *tbs17/MathBert* (с замороженным backbone и без заморозки), проанализируйте результаты. Сравните скоры с первым заданием. Получилось лучше или нет? Почему?

### Задание 6 (1 балл)

Напишите функцию для отрисовки карт внимания первого слоя для моделей из задания

In [ ]:
import matplotlib.pyplot as plt
import torch
from transformers import AutoTokenizer

import matplotlib.pyplot as plt

def draw_first_layer_attention_maps(attention_head_ids, text, model, tokenizer):
    # Подготовка входных данных
    inputs = tokenizer(text, return_tensors="pt")
    input_ids = inputs['input_ids'].to(model.device)  # Убедитесь, что тензоры перенесены на нужное устройство
    attention_mask = inputs['attention_mask'].to(model.device)

    # Установка модели в режим оценки
    model.eval()

    # Включаем вычисление внимания
    with torch.no_grad():
        # Получаем внимание
        outputs = model(input_ids, attention_mask=attention_mask)
        attentions = outputs['attentions']

    # Выбор первого слоя внимания
    first_layer_attention = attentions[0]  # (batch_size, num_heads, seq_length, seq_length)
    num_heads = first_layer_attention.size(1)

    # Получаем токены для подписей
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])

    # Визуализация карт внимания для указанных голов
    fig, axes = plt.subplots(1, len(attention_head_ids), figsize=(20, 10))

    if len(attention_head_ids) == 1:
        axes = [axes]  # Обеспечиваем итерируемость для одного элемента

    for i, head_id in enumerate(attention_head_ids):
        # Проверка на корректность номера головы
        if head_id < 0 or head_id >= num_heads:
            raise ValueError(f"Head id {head_id} is out of valid range [0, {num_heads-1}]")

        # Перемещаем тензор на CPU перед конвертацией в NumPy
        attention = first_layer_attention[0, head_id].cpu().numpy()

        ax = axes[i]
        cax = ax.matshow(attention, cmap='viridis')
        fig.colorbar(cax, ax=ax)
        ax.set_xticks(range(len(tokens)))
        ax.set_yticks(range(len(tokens)))
        ax.set_xticklabels(tokens, rotation=90)
        ax.set_yticklabels(tokens)
        ax.set_title(f'Head {head_id}')

    plt.tight_layout()
    plt.show()



In [ ]:
from transformers import AutoTokenizer

# Используйте правильное название предобученной модели для токенизатора
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
text = "раз два три четыре."
# text = df['Задача'].sample(1).tolist() 

model = rubert_tiny_full_finetuned # rubert_tiny_finetuned_with_freezed_backbone 
attention_head_ids = [0, 3]  # Головы внимания, которые мы хотим визуализировать

draw_first_layer_attention_maps(attention_head_ids, text, model, tokenizer)

### Задание 7 (1 балл)

Проведите инференс для всех моделей **ДО ДООБУЧЕНИЯ** на 2-3 текстах из датасета. Посмотрите на головы Attention первого слоя в каждой модели на выбранных текстах (отрисуйте их отдельно).

Попробуйте их проинтерпретировать. Какие связи улавливают карты внимания? (если в модели много голов Attention, то проинтерпретируйте наиболее интересные)

In [ ]:
from transformers import AutoModel, AutoTokenizer

# Загрузка токенизатора
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")

# Инициализация модели
model_rubert_tiny = AutoModel.from_pretrained("cointegrated/rubert-tiny2", output_attentions=True)
model_rubert_tiny.eval()  # Перевести модель в режим оценки


texts = df['Задача'].sample(3).tolist()  # Случайный выбор трех текстов

import matplotlib.pyplot as plt

def visualize_attention(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        attentions = outputs.attentions  # Получаем внимания

    # Визуализируем первый слой (первые две головы внимания)
    attention = attentions[0][0, :2].detach().cpu().numpy()  # Первый слой, первые две головы
    tokens = tokenizer.convert_ids_to_tokens(input_ids[0])

    fig, axs = plt.subplots(1, 2, figsize=(20, 10))
    for i, ax in enumerate(axs):
        ax.matshow(attention[i], cmap='viridis')
        ax.set_xticks(range(len(tokens)))
        ax.set_yticks(range(len(tokens)))
        ax.set_xticklabels(tokens, rotation=90)
        ax.set_yticklabels(tokens)
        ax.set_title(f'Head {i+1} of Layer 1')

    plt.show()

for text in texts:
    visualize_attention(text, model_rubert_tiny, tokenizer)



### Задание 8 (1 балл)

Сделайте то же самое для дообученных моделей. Изменились ли карты внимания и связи, которые они улавливают? Почему?

In [ ]:
### YOUR CODE IS HERE